In [1]:
import os

In [2]:
%pwd

'c:\\Users\\howar\\Documents\\UIUC\\IE 421\\project\\group_10_project\\mlops\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\howar\\Documents\\UIUC\\IE 421\\project\\group_10_project\\mlops'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class PrepareModelConfig:
    root_dir: Path
    model_path: Path
    params_input_size: int
    params_hidden_size: int
    params_num_layers: int
    params_fc_layers_size: int
    params_learning_rate: float

In [11]:
from src.lstm_price_prediction.constants import *
from src.lstm_price_prediction.utils.common import read_yaml, create_directories

ModuleNotFoundError: No module named 'src.lstm_price_prediction'

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    def get_prepare_model_config(self) -> PrepareModelConfig:
        config = self.config.prepare_model
        
        create_directories([config.root_dir])

        prepare_model_config = PrepareModelConfig(
            root_dir=Path(config.root_dir),
            model_path=Path(config.model_path),
            params_input_size = self.params.INPUT_SIZE,
            params_hidden_size = self.params.HIDDEN_SIZE,
            params_num_layers = self.params.NUM_LAYERS,
            params_fc_layers_size = self.params.FC_LAYERS_SIZE,
            params_learning_rate = self.params.LEARNING_RATE
   
        )

        return prepare_model_config

In [8]:
import os
import urllib.request as request
from zipfile import ZipFile
import torch
from src.lstm_price_prediction.components.rnn_model import rnn_models

In [9]:
class PrepareModel:
    def __init__(self, config: PrepareModelConfig):
        self.config = config
        self.device  = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    def get_model(self):
        self.model = rnn_models.LSTMModel(self.config.params_input_size,self.config.params_hidden_size ,self.config.params_num_layers, self.config.params_fc_layers_size).to(self.device)
        self.save_model(path=self.config.model_path, model=self.model)
        


    @staticmethod
    def save_model(path: Path, model: torch.nn.Module):
        torch.save(model.state_dict(), path)




In [10]:
try:
    config = ConfigurationManager()
    prepare_base_model_config = config.get_prepare_model_config()
    prepare_base_model = PrepareModel(config=prepare_base_model_config)
    prepare_base_model.get_model()
except Exception as e:
    raise e

config\config.yaml
[2024-05-11 04:38:53,415: INFO: common: yaml file: config\config.yaml loaded successfully]
params.yaml
[2024-05-11 04:38:53,418: INFO: common: yaml file: params.yaml loaded successfully]
[2024-05-11 04:38:53,419: INFO: common: created directory at: artifacts]
[2024-05-11 04:38:53,419: INFO: common: created directory at: artifacts/prepare_model]
